In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
train_file=pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
test_file=pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')

In [3]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text_id      3911 non-null   object 
 1   full_text    3911 non-null   object 
 2   cohesion     3911 non-null   float64
 3   syntax       3911 non-null   float64
 4   vocabulary   3911 non-null   float64
 5   phraseology  3911 non-null   float64
 6   grammar      3911 non-null   float64
 7   conventions  3911 non-null   float64
dtypes: float64(6), object(2)
memory usage: 244.6+ KB


In [4]:
train_file.head(3)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5


In [5]:
test_file.head(2)

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...


In [6]:
train_file.columns

Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions'],
      dtype='object')

In [7]:
labels=['cohesion', 'syntax', 'vocabulary','phraseology', 'grammar', 'conventions']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [8]:
# from transformers import AutoTokenizer
# import numpy as np

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# def preprocess_data(examples):
#   # take a batch of texts
#     text = train_file["full_text"]
#     # encode them
#     encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     # add labels
#     labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
#     # create numpy array of shape (batch_size, num_labels)
#     labels_matrix = np.zeros((len(text), len(labels)))
#     # fill numpy array
#     for idx, label in enumerate(labels):
#         labels_matrix[:, idx] = labels_batch[label]
#         encoding["labels"] = labels_matrix.tolist()

#     return encoding

In [9]:
# MODEL_MAX_LENGTH=160

# def bert_encode(data,maximum_length) :
#     input_ids = []
#     attention_masks = []
  

#     for i in range(len(data.full_text)):
#         encoded = tokenizer.encode_plus(

#         data.full_text[i],
#         add_special_tokens=True,
#         max_length=maximum_length,
#         pad_to_max_length=True,

#         return_attention_mask=True,

#         )

#         input_ids.append(encoded['input_ids'])
#         attention_masks.append(encoded['attention_mask'])
        
#     return np.array(input_ids),np.array(attention_masks)

# train_input_ids,train_attention_masks = bert_encode(train_file[['full_text']],MODEL_MAX_LENGTH)
# test_input_ids,test_attention_masks = bert_encode(test_file[['full_text']],MODEL_MAX_LENGTH)

In [10]:
# from transformers import AutoModel
# model = AutoModel.from_pretrained("bert-base-uncased", num_labels=6)

In [11]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_file['full_text'])
X_test = vectorizer.transform(test_file['full_text'])

In [13]:
y=train_file[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]
# y_test=test_file[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]

In [14]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [15]:
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train, y)

MultiOutputRegressor(estimator=LGBMRegressor(n_estimators=1000, verbose=-1))

In [16]:
test_preds = model.predict(X_test)

In [17]:
test_file.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [18]:
sample = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')
sample.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [19]:
sample['cohesion']=test_preds[:,0]
sample['syntax']=test_preds[:,1]
sample['vocabulary']=test_preds[:,2]
sample['phraseology']=test_preds[:,3]
sample['grammar']=test_preds[:,4]
sample['conventions']=test_preds[:,5]

In [20]:
sample.to_csv('submission.csv',index=False)